In [5]:
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup as bs
from random import shuffle
from re import sub

In [6]:
def get_soup(url):
    '''url을 받아 소스코드를 긁어온 후 
    BeautifulSoup로 파싱한 것을 반환합니다.
    '''
    with request.urlopen(url) as f:
        html = f.read().decode('utf-8')
    soup = bs(html, 'html5lib')
    return soup

In [7]:
def get_pets(soup):
    '''BeautifulSoup로 파싱된 소스코드를 받아 
    해당 페이지의 하단 청원목록 항목들을 리스트로 반환합니다
    '''
    pet_body = soup.select("div.bl_body ul")
    pet_list = pet_body[1]
    pet_items = pet_list.select("li div.bl_wrap")
    return pet_items

In [8]:
def get_text(elem, choice):
    '''청원목록 항목 내의 태그(elem)와 노드 위치(choice)를 받아
    태그의 텍스트 노드를 찾아 텍스트를 반환합니다. 
    '''
    elem_children = list(elem.children)
    elem_text = elem_children[choice]
    elem_trim = elem_text.strip()
    return elem_trim

In [9]:
def get_item_info(item, choice=[1, 1, 1, 2]):
    '''청원목록의 한 항목을 받아 
    항목의 번호, 분류, 제목, 참여인원을 리스트로 반환합니다.
    잘못된 텍스트 노드가 반환될 경우 choice를 0|1|2로 직접 설정해보시기 바랍니다. 
    '''
    num_choice, category_choice, title_choice, votes_choice = choice
    num_raw = item.select("div.bl_no")[0]
    category_raw = item.select("div.bl_category")[0]
    title_raw = item.select("div.bl_subject a")[0]
    votes_raw = item.select("div.bl_agree")[0]
    
    item_info = [get_text(num_raw, num_choice), get_text(category_raw, category_choice), 
                 get_text(title_raw, title_choice), get_text(votes_raw, votes_choice)]
    return item_info


In [10]:
def crawl_page(url, mode='df'):
    '''청원목록 페이지 url을 받아 
    페이지의 항목(15개)를 추출해 데이터프레임 형태로 반환합니다.
    mode='list'를 parameter로 입력할 시 데이터프레임 대신 리스트를 반환합니다.
    '''
    soup = get_soup(url)
    pet_items = get_pets(soup)
    lst = []
    for i in pet_items:
        lst.append(get_item_info(i))
        
    if mode == 'df':    
        df = pd.DataFrame(lst, columns=['번호', '분류', '제목', '참여인원'])
        df['참여인원'] = df['참여인원'].str.replace('명', '').astype("int64") 
        return df
    else: 
        return lst

In [11]:
def get_top_id():
    '''가장 최근에 등록된 청원의 번호(id)를 int로 반환합니다
    '''
    url = "https://www1.president.go.kr/petitions"
    soup = get_soup(url)
    pet_top = get_pets(soup)[0]
    pet_top_id = int(get_item_info(pet_top)[0])
    return pet_top_id

In [12]:
def crawl_pet(petnum):
    '''청원글 본문 번호를 받아 해당 청원글의 제목과 본문을 추출,
    리스트로 반환.
    '''
    url = "https://www1.president.go.kr/petitions/" + str(petnum)
    soup = get_soup(url)
    title = soup.find("h3", class_="petitionsView_title").text
    content_raw = soup.find("div", class_="View_write").text
    content = sub("\n\t+", '', content_raw)
    return ([title, content])

# 기본과제

국민청원 첫 페이지 하단의 **청원 목록**에서 **번호, 분류, 제목, 참여인원**을 추출하여 **DataFrame**에 담아주세요.

In [9]:
url = "https://www1.president.go.kr/petitions"
df = crawl_page(url)
df 

,번호,분류,제목,참여인원
0,244595,정치개혁,언론에서 8만명 강조하고있다 이재명 관련,0
1,244594,행정,인력사무소 여행사 환전소 에대한 청산이필요,0
2,244593,정치개혁,제발 뒤로가기좀 해주세요..아~~진짜 답답하다...,0
3,244592,정치개혁,이재명경기도지사 청원 참여 가장 많은 곳으로 링크 ...,7
4,244591,반려동물,반려동물 키우는 사람들!!!,1
5,244590,정치개혁,한 국회의원을 보낸 국회와 정치계에 청원함,0
6,244589,기타,자동차 led규제 개선,4
7,244588,행정,폭염으로 인한 제안,1
8,244587,인권/성평등,유해드라마 미스터 선샤인 방영금지 해주세요,0
9,244586,기타,쇄빙선건조,0


# 선택과제 1

국민청원 목록 첫 페이지 주소는 https://www1.president.go.kr/petitions?page=1 입니다. 두번째 페이지 주소는 https://www1.president.go.kr/petitions?page=2 입니다. **처음 10개 페이지**에 순차적으로 방문하여 **번호, 분류, 제목, 참여인원**을 추출하고 그 결과를 **DataFrame**에 담아주세요.

애란쌤 피드백을 참조해서 데이터프레임을 만드는 방식을 바꾸어 보았습니다.

> Iteratively appending rows to a DataFrame can be more computationally intensive than a single concatenate. A better solution is to append those rows to a list and then concatenate the list with the original DataFrame all at once.

- 첫 번째 코드는 제가 처음 만들었던, 빈 데이터프레임을 우선 생성한 후 루프를 돌리며 크롤링한 데이터를 매번 DataFrame타입으로 변환하여 반복적으로  DataFrame에 append 하는 방식을 사용했으며

- 두 번째 코드는 애란쌤 피드백을 참조해서 빈 리스트를 우선 생성한 후 루프를 돌리며 크롤링한 데이터를 리스트에 append해 다 크롤링한 후 DataFrame으로 만들어주는 방식을 사용했습니다.


어느쪽이 더 효율적인지 측정하기 위해 처리에 걸리는 시간을 비교해봅니다.

In [13]:
from time import time

In [16]:
# PLAN 1
tic = time()

url_base = "https://www1.president.go.kr/petitions?page="
df = pd.DataFrame([], columns=['번호', '분류', '제목', '참여인원'])

for i in range(1,11):
    url_temp = url_base + str(i)
    df_temp = crawl_page(url_temp)
    df = df.append(df_temp, ignore_index=True)
    print("페이지 %d를 크롤링 중입니다"%i)
    
print("크롤링 완료.")    

toc = time()
print('걸린 시간: ', toc - tic)

페이지 1를 크롤링 중입니다
페이지 2를 크롤링 중입니다
페이지 3를 크롤링 중입니다
페이지 4를 크롤링 중입니다
페이지 5를 크롤링 중입니다
페이지 6를 크롤링 중입니다
페이지 7를 크롤링 중입니다
페이지 8를 크롤링 중입니다
페이지 9를 크롤링 중입니다
페이지 10를 크롤링 중입니다
크롤링 완료.
걸린 시간:  7.343382358551025


In [17]:
# PLAN 2
tic = time()

url_base = "https://www1.president.go.kr/petitions?page="
article_lst = []

for i in range(1,11):
    url_temp = url_base + str(i)
    df_temp = crawl_page(url_temp, mode="list")[1: ]
    article_lst.append(df_temp)
    print("페이지 %d를 크롤링 중입니다"%i)

df = pd.DataFrame(article_lst[0], columns=['번호', '분류', '제목', '참여인원'])
df['참여인원'] = df['참여인원'].str.replace('명', '').astype("int64")
print("크롤링 완료.")    

toc = time()
print('걸린 시간: ', toc - tic)

페이지 1를 크롤링 중입니다
페이지 2를 크롤링 중입니다
페이지 3를 크롤링 중입니다
페이지 4를 크롤링 중입니다
페이지 5를 크롤링 중입니다
페이지 6를 크롤링 중입니다
페이지 7를 크롤링 중입니다
페이지 8를 크롤링 중입니다
페이지 9를 크롤링 중입니다
페이지 10를 크롤링 중입니다
크롤링 완료.
걸린 시간:  7.555988311767578


In [11]:
print(df.shape)
df.sample(15)

(150, 4)


,번호,분류,제목,참여인원
95,244502,성장동력,연금통합,1
56,244541,정치개혁,노회찬의원을 살려내라.,5
149,244448,안전/환경,화재안전특별조사의 자료정리방식에 관하여,0
43,244554,교통/건축/국토,전기 누진세제도 폐지 부탁합니다.,13
116,244481,보건복지,"저소득층 국민을 위한 폭서기, 폭한기 대책을 청원합 ...",2
39,244558,보건복지,장애인차량 주차 표지판 개선을 부탁 드립니다,11
120,244477,기타,전기요금 누진제 폐지,14
12,244584,인권/성평등,일간베스트 사이트 조사및 폐지,11
62,244535,기타,국립경찰병원의료진개선,0
29,244568,정치개혁,붕괴3rd 한국서버를 중국서버로 이전해주세요,2


# 선택과제 2

2018년 7월 17일 현재 국민청원의 글 번호는 1번부터 238663번까지 부여되어 있습니다. 하지만 중간중간 *삭제된 청원*이 있어서 실제 청원 수는 238663개보다 적습니다. 309510번 청원글 본문의 URL은 https://www1.president.go.kr/petitions/309510 입니다.
URL에서 글번호만 바꾸면 해당 청원의 본문으로 이동할 수 있습니다. **이러한 URL 패턴의 특성을 활용**하여 전체 청원 중 **임의로 100개** 청원의 **제목과 본문**을 추출하여 **DataFrame**에 담아주세요.

In [0]:
url_base = "https://www1.president.go.kr/petitions/"
topid = get_top_id()
pool = list(range(1, topid+1))
shuffle(pool)
picked = []
article_lst = []

while len(picked) < 100:
    pet = pool.pop()
    try:
        article = crawl_pet(pet)
        article_lst.append(article)
        picked.append(pet)
    except:
        print('삭제된 청원입니다. 글 번호: %d, 총 크롤링 수: %d'%(pet, len(article_lst)) )
    
df = pd.DataFrame(article_lst, columns=["제목", "본문"])
print("청원 크롤링이 완료되었습니다. 총 크롤링 수: %d" %len(df))

삭제된 청원입니다. 글 번호: 241278, 총 크롤링 수: 0
삭제된 청원입니다. 글 번호: 221304, 총 크롤링 수: 6
삭제된 청원입니다. 글 번호: 203122, 총 크롤링 수: 12
삭제된 청원입니다. 글 번호: 234467, 총 크롤링 수: 12
삭제된 청원입니다. 글 번호: 117660, 총 크롤링 수: 18
삭제된 청원입니다. 글 번호: 87820, 총 크롤링 수: 18
삭제된 청원입니다. 글 번호: 225492, 총 크롤링 수: 18
삭제된 청원입니다. 글 번호: 86934, 총 크롤링 수: 26
삭제된 청원입니다. 글 번호: 233178, 총 크롤링 수: 26
삭제된 청원입니다. 글 번호: 8565, 총 크롤링 수: 30
삭제된 청원입니다. 글 번호: 77870, 총 크롤링 수: 35


In [13]:
print(df.shape)
df.sample(10)

(100, 2)


,제목,본문
5,문체부에서 '리메즈 엔터테이먼트'의 음원순위조작 의혹을 해소해주세요,4월 12 오전 1시경에 가수 '닐로' 의 지나오다의 곡이 음원최대 사이트 멜론에서...
43,청소년보호법폐지,청원합니다
3,강원랜드 특검 드루킹특검 이유미조작사건 특검다같이합시다,국회의원들 특히 자한당 바미당이좋아하는 특검 국민들이 의구심있는 강원랜드 이유...
95,이명박근혜 패거리들의 천인공노 할 만행을 낱낱히 밝혀야 나라가 바로섭니다,나라 말아 먹고도 여기저기 싸돌아 댕기는 이명박 출국금지
39,자유한국당정신치매해산,"안녕하세요^^시민들성질폭발합니다.노무현재단괴롭다,문재인님너무힘들요,국가운영너무잘함,..."
20,청소년 보호법 폐지 꼭 이루어졌으면,"저 5명때문에 다른 청소년들이 피해를 받을 것을 생각하면 좀 그렇지만, 걔네들 5명..."
46,지방선거떄 보자,현 정부가 촛불로 탄생한 이유가 니들이 잘나서가 아니라 저들이 못 나서 그런건데 현...
10,모든 범죄에 대한 형벌의 기준을 바로 잡아 주세요. (년 이하 -> 년 이상),"우리나라의 형벌 기준은 ~년 이하의 징역, ~만원 이하의 벌금 등으로 이루어져 있습..."
63,"이번 개헌 때 ""대통령 사면권 폐지"" 청원합니다","대한민국 대통령은 ""사면권""이라는 특별한 권한을 가지고 있습니다 대한민국은 법치국가..."
37,염순덕상사의 죽음에 대해 진실을 밝혀주세요,4월 24일과 31일에 방영된 sbs그것이알고싶다를 통해 세상에 알려진 억울하게 돌...
